# Rede neural convolucional para classificação binária de imagens (loader)

In [ ]:
import pandas as pd
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import seaborn as sns
import sklearn.metrics
import matplotlib.pyplot as plt

In [ ]:
test_path = 'dataset1/test'

Carregamento do modelo pré-treinado

In [ ]:
model = load_model('model.33-0.14.hdf5')

In [ ]:
model.summary()

Carregamento das imagens de teste

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./225)
test_generator = test_datagen.flow_from_directory('dataset1/test',
                                                 target_size = (150,150),
                                                 batch_size = 1,
                                                    shuffle=False)

Reset do gerador de testes e predição

In [ ]:
test_generator.reset()
pred=model.predict_generator(test_generator,verbose=1)

In [ ]:
concl = np.empty([len(pred)],  dtype=object)
real = np.empty([len(pred)],  dtype=object)
for i in range(len(pred)) :
    if(pred[i] > 0.5):
        concl[i] = 'Infectado'
    else:
        concl[i] = 'Desinfectado'
        
for i in range(len(pred)):
    if(i <= (len(pred)/2)-1):                    #Caso seja par o número de amostras, se ímpar, aterar aqui
        real[i] = 'Desinfectado'
    else:
        real[i] = 'Infectado'

Gera arquivo do Excel com conclusões

In [ ]:
filenames=test_generator.filenames
for k in range(len(filenames)):
    filenames[k] = filenames[k].replace('test_folder\\', '')

In [ ]:
results=pd.DataFrame({"Nome do arquivo":filenames,
                      "Conclusão":concl})
results.to_excel("resultados.xlsx",index=False, encoding='utf-8')

In [ ]:
def cm_analysis(y_true, y_pred, filename, labels, ymap=None, figsize=(10,10)):
    """
    Generate matrix plot of confusion matrix with pretty annotations.
    The plot image is saved to disk.
    args: 
      y_true:    true label of the data, with shape (nsamples,)
      y_pred:    prediction of the data, with shape (nsamples,)
      filename:  filename of figure file to save
      labels:    string array, name the order of class labels in the confusion matrix.
                 use `clf.classes_` if using scikit-learn models.
                 with shape (nclass,).
      ymap:      dict: any -> string, length == nclass.
                 if not None, map the labels & ys to more understandable strings.
                 Caution: original y_true, y_pred and labels must align.
      figsize:   the size of the figure plotted.
    """
    if ymap is not None:
        y_pred = [ymap[yi] for yi in y_pred]
        y_true = [ymap[yi] for yi in y_true]
        labels = [ymap[yi] for yi in labels]
    cm = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=labels)
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=labels, columns=labels)
    cm.index.name = 'Real'
    cm.columns.name = 'Previsão'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, annot=annot, fmt='', ax=ax, cmap="YlGnBu")
    ax.set_title('Matriz Confusão')
    ax.yaxis.set_ticklabels(labels, va = 'center')
    plt.savefig(filename, format = 'pdf')
    
cm_analysis(real, concl, 'confusao.pdf', ['Desinfectado','Infectado'])